#**PROYECTO KAGGLE - Pruebas Saber Pro Colombia**

Modelo Predictivo a partir de la tabla de datos train.csv sobre el rendimiento de los estudiantes en las pruebas Saber Pro.

Hecho por:
- Juan Manuel Areiza Ospina - C.C. 1018226898
- Samuel Puerta Patiño - C.C. 1023624795
- Brayan Stiven Gómez Villa - C.C 1018224235



Se implementa un modelo de clasificación para predecir el rendimiento global de estudiantes en las Pruebas Saber Pro de Colombia, utilizando XGBoost con optimización de hiperparámetros mediante Optuna y pseudo-etiquetado.

El contexto del proyecto es predecir el desempeño de estudiantes (bajo, medio-bajo, medio-alto, alto) basado en datos socioeconómicos, institucionales y estadísticos.

## Importaciones

En esta sección se importan las bibliotecas necesarias para el procesamiento de datos, modelado y optimización. Pandas y NumPy para manejo de datos, Optuna para optimización de hiperparámetros, XGBoost como modelo de clasificación, y herramientas de scikit-learn para validación cruzada y preprocesamiento.

In [ ]:
pip install pandas numpy optuna xgboost scikit-learn

In [ ]:
import pandas as pd
import numpy as np
import optuna
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder
from pandas.api.types import CategoricalDtype
import os

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 848MB/s]


In [ ]:
!unzip udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip

Archive:  udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
  inflating: submission_example.csv  
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               


## Configuración

Aquí se definen las constantes y configuraciones del script, como los nombres de archivos, la columna objetivo, el estado aleatorio para reproducibilidad, el número de pruebas de Optuna y el umbral para pseudo-etiquetado.

In [ ]:
# --- Configuration ---
TRAIN_FILE = "train.csv"
TEST_FILE = "test.csv"
SUBMISSION_FILE = "submission_advanced_v2.csv"
TARGET_COL = "RENDIMIENTO_GLOBAL"
RANDOM_STATE = 42
N_TRIALS = 10  # Increased to 20
PSEUDO_LABEL_THRESHOLD = 0.90 # Back to 0.90 for safety

## Funciones Auxiliares

Estas funciones ayudan en la carga de datos, limpieza de texto y ingeniería de características. Son cruciales para preparar los datos antes del modelado.

### Función load_data

Esta función carga los datos de entrenamiento y prueba desde los archivos CSV especificados. Verifica que el archivo de entrenamiento exista y devuelve los DataFrames.

In [ ]:
def load_data():
    print("Loading data...")
    if not os.path.exists(TRAIN_FILE):
        raise FileNotFoundError(f"Train file not found at {os.path.abspath(TRAIN_FILE)}")

    train_df = pd.read_csv(TRAIN_FILE)
    test_df = pd.read_csv(TEST_FILE)
    return train_df, test_df

### Función clean_text

Esta función limpia y estandariza las columnas de texto en el DataFrame. Elimina espacios en blanco, estandariza respuestas Sí/No y corrige inconsistencias en el texto.

In [ ]:
def clean_text(df):
    # Standardize text columns
    df.columns = df.columns.str.strip()
    # Map all string columns to strip whitespace
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].astype(str).str.strip()

    # Standardize Yes/No
    replace_dict = {
        "Si": "Sí", "si": "Sí", "sÃ­": "Sí",
        "N": "No", "NO": "No", "n": "No"
    }
    df.replace(replace_dict, inplace=True)
    return df

### Función feature_engineering

Esta función realiza la ingeniería de características: elimina la columna ID, maneja la alta cardinalidad en programas académicos, codifica ordinalmente variables categóricas, crea índices socioeconómicos y de educación parental, y calcula presión financiera. También elimina columnas ruidosas.

In [ ]:
def feature_engineering(df, is_train=True, allowed_programs=None):
    print("Engineering features...")
    df = df.copy()

    # 1. Drop ID
    if 'ID' in df.columns:
        df = df.drop(columns=['ID'])

    # 2. Handle E_PRGM_ACADEMICO (High Cardinality)
    if 'E_PRGM_ACADEMICO' in df.columns:
        if is_train:
            frecuencias = df["E_PRGM_ACADEMICO"].value_counts()
            allowed_programs = set(frecuencias.head(200).index) # Increased to 250 to match best model
            df["E_PRGM_ACADEMICO"] = df["E_PRGM_ACADEMICO"].apply(lambda x: x if x in allowed_programs else "OTROS_PROGRAMAS")
        else:
            if allowed_programs is not None:
                df["E_PRGM_ACADEMICO"] = df["E_PRGM_ACADEMICO"].apply(lambda x: x if x in allowed_programs else "OTROS_PROGRAMAS")

    # 3. Ordinal Encoding & Numeric Conversion

    # Hours Worked
    orden_horas = ['0', "Menos de 10 horas", "Entre 11 y 20 horas", "Entre 21 y 30 horas", "Más de 30 horas"]
    horas_map = {k: i for i, k in enumerate(orden_horas)}
    if "E_HORASSEMANATRABAJA" in df.columns:
        df["E_HORASSEMANATRABAJA_NUM"] = df["E_HORASSEMANATRABAJA"].map(horas_map).fillna(0)

    # Education Level
    orden_educacion = [
        "Ninguno", "No sabe", "No Aplica", "Primaria incompleta", "Primaria completa",
        "Secundaria (Bachillerato) incompleta", "Secundaria (Bachillerato) completa",
        "Técnica o tecnológica incompleta", "Técnica o tecnológica completa",
        "Educación profesional incompleta", "Educación profesional completa", "Postgrado"
    ]
    # Custom mapping to give "Postgrado" higher value
    edu_map = {k: i for i, k in enumerate(orden_educacion)}

    for col in ["F_EDUCACIONPADRE", "F_EDUCACIONMADRE"]:
        if col in df.columns:
            df[f"{col}_NUM"] = df[col].map(edu_map).fillna(-1)

    # Estrato
    if "F_ESTRATOVIVIENDA" in df.columns:
        # Extract number from "Estrato 1", etc.
        df["F_ESTRATOVIVIENDA_NUM"] = df["F_ESTRATOVIVIENDA"].astype(str).str.extract('(\d+)').astype(float).fillna(0)

    # 4. New Interaction Features

    # Socioeconomic Index
    # Map Yes/No to 1/0
    yes_no_cols = ['F_TIENEINTERNET', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'F_TIENECOMPUTADOR']
    for col in yes_no_cols:
        if col in df.columns:
            df[f"{col}_NUM"] = df[col].apply(lambda x: 1 if x == 'Sí' else 0)

    # Combine duplicates if F_TIENEINTERNET.1 exists
    if 'F_TIENEINTERNET.1' in df.columns:
        df['F_TIENEINTERNET_COMBINED'] = df.apply(lambda row: 1 if (row.get('F_TIENEINTERNET') == 'Sí' or row.get('F_TIENEINTERNET.1') == 'Sí') else 0, axis=1)
    elif 'F_TIENEINTERNET' in df.columns:
        df['F_TIENEINTERNET_COMBINED'] = df['F_TIENEINTERNET_NUM']
    else:
        df['F_TIENEINTERNET_COMBINED'] = 0

    # Create Index
    df['SOCIOECONOMIC_INDEX'] = (
        df.get('F_ESTRATOVIVIENDA_NUM', 0) +
        df.get('F_TIENELAVADORA_NUM', 0) +
        df.get('F_TIENEAUTOMOVIL_NUM', 0) +
        df.get('F_TIENECOMPUTADOR_NUM', 0) +
        df.get('F_TIENEINTERNET_COMBINED', 0)
    )

    # Parental Education Index
    df['PARENT_EDU_INDEX'] = (df.get('F_EDUCACIONPADRE_NUM', 0) + df.get('F_EDUCACIONMADRE_NUM', 0)) / 2

    # Financial Pressure Proxy (Tuition * Work Hours)
    # Matricula is usually categorical ranges, let's map roughly
    matricula_map = {
        "No pagó matrícula": 0,
        "Menos de 500 mil": 250000,
        "Entre 500 mil y menos de 1 millón": 750000,
        "Entre 1 millón y menos de 2.5 millones": 1750000,
        "Entre 2.5 millones y menos de 4 millones": 3250000,
        "Entre 4 millones y menos de 5.5 millones": 4750000,
        "Entre 5.5 millones y menos de 7 millones": 6250000,
        "Más de 7 millones": 8000000
    }
    if "E_VALORMATRICULAUNIVERSIDAD" in df.columns:
        df["VALOR_MATRICULA_EST"] = df["E_VALORMATRICULAUNIVERSIDAD"].map(matricula_map).fillna(0)
        df["FINANCIAL_PRESSURE"] = df["VALOR_MATRICULA_EST"] * df.get("E_HORASSEMANATRABAJA_NUM", 0)

    # 5. DROP NOISY COLUMNS (Crucial Step from Best Model)
    # We used these to create indices, now we drop the raw ones that were deemed noisy
    cols_to_drop = [
        'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL',
        'E_PRIVADO_LIBERTAD', 'F_TIENEINTERNET.1',
        'INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4',
        # Also drop the intermediate numeric cols we created if they are redundant,
        # but let's keep the _NUM ones as they are better than raw text.
        # We drop the original text ones if we have _NUM
    ]
    # Actually, let's strictly follow the best model's drop list + our used raw inputs
    # Best model dropped: 'E_VALORMATRICULAUNIVERSIDAD', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD', 'E_PAGOMATRICULAPROPIO', 'F_TIENEINTERNET.1', 'INDICADOR_1'...'INDICADOR_4'

    final_drop = [c for c in cols_to_drop if c in df.columns]
    df = df.drop(columns=final_drop)

    return df, allowed_programs

<>:44: SyntaxWarning: invalid escape sequence '\d'
<>:44: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-3484362777.py:44: SyntaxWarning: invalid escape sequence '\d'
  df["F_ESTRATOVIVIENDA_NUM"] = df["F_ESTRATOVIVIENDA"].astype(str).str.extract('(\d+)').astype(float).fillna(0)


## Función Principal (main)

Esta es la función principal que ejecuta todo el flujo: carga los datos, los limpia, realiza ingeniería de características, prepara los datos para el modelo, optimiza hiperparámetros con Optuna, aplica pseudo-etiquetado, hace predicciones finales y guarda los resultados.

### Optimización con Optuna

Se define una función objetivo para Optuna que prueba diferentes hiperparámetros del modelo XGBoost usando validación cruzada estratificada. El objetivo es maximizar la precisión.

### Pseudo-Etiquetado

Se entrena un modelo inicial con los mejores parámetros, se predicen probabilidades en el conjunto de prueba, y se seleccionan muestras de alta confianza para agregar al conjunto de entrenamiento, mejorando el modelo.

### Predicción Final y Guardado

Se generan las predicciones finales en el conjunto de prueba, se decodifican las etiquetas, se crea el archivo de envío y se muestra la importancia de las características.

In [ ]:
def main():
    # 1. Load
    train_df_raw, test_df_raw = load_data()
    test_ids = test_df_raw['ID']

    # 2. Clean & Preprocess
    train_df_clean = clean_text(train_df_raw)
    test_df_clean = clean_text(test_df_raw)

    train_df, allowed_programs = feature_engineering(train_df_clean, is_train=True)
    test_df, _ = feature_engineering(test_df_clean, is_train=False, allowed_programs=allowed_programs)

        # Prepare X, y
    X = train_df.drop(columns=[TARGET_COL], errors='ignore')
    y_raw = train_df[TARGET_COL]
    X_test = test_df.drop(columns=['ID'], errors='ignore')

    # Align columns
    missing_cols = set(X.columns) - set(X_test.columns)
    for c in missing_cols:
        X_test[c] = 0
    X_test = X_test[X.columns]

    # Encode Target
    le = LabelEncoder()
    y = le.fit_transform(y_raw)

    # Convert object columns to category for XGBoost
    categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
    print(f"Categorical columns: {categorical_cols}")
    for col in categorical_cols:
        X[col] = X[col].astype("category")
        X_test[col] = X_test[col].astype("category")

    # 3. Optuna Optimization
    def objective(trial):
        param = {
            'objective': 'multi:softmax',
            'num_class': len(le.classes_),
            'tree_method': 'hist',
            'enable_categorical': True,
            'eval_metric': 'mlogloss',
            'random_state': RANDOM_STATE,
            'n_jobs': -1,

            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
            'max_depth': trial.suggest_int('max_depth', 4, 10),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 5),
            'reg_lambda': trial.suggest_float('reg_lambda', 0.1, 5),
            'n_estimators': trial.suggest_int('n_estimators', 200, 600),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 5)
        }

        cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)
        model = xgb.XGBClassifier(**param)
        scores = cross_val_score(model, X, y, cv=cv, scoring='accuracy')
        return scores.mean()

    print(f"Starting Optuna with {N_TRIALS} trials...")
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=N_TRIALS)

    print(f"Best CV Accuracy: {study.best_value:.4f}")
    best_params = study.best_params

    # 4. Pseudo-Labeling
    print("\n--- Pseudo-Labeling Phase ---")

    # Train initial best model
    model_params = best_params.copy()
    model_params.update({
        'objective': 'multi:softmax',
        'num_class': len(le.classes_),
        'tree_method': 'hist',
        'enable_categorical': True,
        'random_state': RANDOM_STATE,
        'n_jobs': -1
    })

    initial_model = xgb.XGBClassifier(**model_params)
    initial_model.fit(X, y)

    # Predict probabilities on test
    probs = initial_model.predict_proba(X_test)
    max_probs = np.max(probs, axis=1)
    preds = np.argmax(probs, axis=1)

    # Select high confidence samples
    high_conf_indices = np.where(max_probs > PSEUDO_LABEL_THRESHOLD)[0]
    print(f"Found {len(high_conf_indices)} pseudo-labels with confidence > {PSEUDO_LABEL_THRESHOLD}")

    if len(high_conf_indices) > 0:
        X_pseudo = X_test.iloc[high_conf_indices]
        y_pseudo = preds[high_conf_indices]

        # Combine with original train
        X_augmented = pd.concat([X, X_pseudo], axis=0)
        y_augmented = np.concatenate([y, y_pseudo], axis=0)

        print(f"Retraining with augmented size: {len(X_augmented)}")
        final_model = xgb.XGBClassifier(**model_params)
        final_model.fit(X_augmented, y_augmented)
    else:
        print("No pseudo-labels added. Using initial model.")
        final_model = initial_model

        # 5. Final Prediction
    print("Generating final predictions...")
    final_preds = final_model.predict(X_test)
    final_preds_decoded = le.inverse_transform(final_preds)

    submission = pd.DataFrame({'ID': test_ids, 'RENDIMIENTO_GLOBAL': final_preds_decoded})
    submission.to_csv(SUBMISSION_FILE, index=False)
    print(f"Saved {SUBMISSION_FILE}")

    # Feature Importance
    importance = final_model.feature_importances_
    feature_names = X.columns
    fi_df = pd.DataFrame({'Feature': feature_names, 'Importance': importance})
    print(fi_df.sort_values(by='Importance', ascending=False).head(20))

if __name__ == "__main__":
    main()

Loading data...
Engineering features...
Engineering features...
Categorical columns: ['E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO', 'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA', 'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE', 'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_EDUCACIONMADRE']


[I 2025-11-28 02:28:30,169] A new study created in memory with name: no-name-03437c30-b4cb-4db8-bbcb-78e9be4918d9


Starting Optuna with 10 trials...


[I 2025-11-28 02:39:49,723] Trial 0 finished with value: 0.4382324891915759 and parameters: {'learning_rate': 0.09674730429833721, 'max_depth': 4, 'subsample': 0.9406613372142963, 'colsample_bytree': 0.913100166784449, 'reg_alpha': 4.509134522158807, 'reg_lambda': 1.6290289661682364, 'n_estimators': 527, 'min_child_weight': 5}. Best is trial 0 with value: 0.4382324891915759.
[I 2025-11-28 02:53:19,232] Trial 1 finished with value: 0.43474368049393736 and parameters: {'learning_rate': 0.15541921878820836, 'max_depth': 7, 'subsample': 0.9204061751765162, 'colsample_bytree': 0.9602712117803982, 'reg_alpha': 4.517519428929507, 'reg_lambda': 2.3200177978757943, 'n_estimators': 359, 'min_child_weight': 3}. Best is trial 0 with value: 0.4382324891915759.
[I 2025-11-28 03:01:03,663] Trial 2 finished with value: 0.43803321120698 and parameters: {'learning_rate': 0.13099655241901237, 'max_depth': 5, 'subsample': 0.8625472807214325, 'colsample_bytree': 0.8228280294857987, 'reg_alpha': 1.342553952

KeyboardInterrupt: 